## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from functools import reduce

## Importing the raw dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filepath = '/content/drive/My Drive/Colab Notebooks/NNDL Project/walmart.csv'
df = pd.read_csv(filepath)
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,7969


In [ ]:
df.shape

(550068, 10)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   User_ID                     550068 non-null  int64 
 1   Product_ID                  550068 non-null  object
 2   Gender                      550068 non-null  object
 3   Age                         550068 non-null  object
 4   Occupation                  550068 non-null  int64 
 5   City_Category               550068 non-null  object
 6   Stay_In_Current_City_Years  550068 non-null  object
 7   Marital_Status              550068 non-null  int64 
 8   Product_Category            550068 non-null  int64 
 9   Purchase                    550068 non-null  int64 
dtypes: int64(5), object(5)
memory usage: 42.0+ MB


In [ ]:
# Checking for missing values
df.isna().sum()

User_ID                       0
Product_ID                    0
Gender                        0
Age                           0
Occupation                    0
City_Category                 0
Stay_In_Current_City_Years    0
Marital_Status                0
Product_Category              0
Purchase                      0
dtype: int64

In [ ]:
# Checking for duplicate values
df.duplicated().sum()

0

In [ ]:
print("No. of users:", df['User_ID'].nunique())
print("No. of products:", df['Product_ID'].nunique())
print("No. of product categories", df['Product_Category'].nunique())

No. of users: 5891
No. of products: 3631
No. of product categories 20


In [ ]:
repeat_purchases = df.groupby(['User_ID', 'Product_ID']).size().reset_index(name='count')
print((repeat_purchases['count'] > 1).sum())
# There are no repeat purchases of a single product

0


In [ ]:
repeat_purchases_in_categories= (
    df.groupby(['User_ID', 'Product_Category'])['Product_ID']
      .nunique()
      .reset_index(name='Num_Products_in_Category')
)

# Filter for cases where the customer bought more than one product in a category
repeat_purchases_in_categories = repeat_purchases_in_categories[repeat_purchases_in_categories['Num_Products_in_Category'] > 1]

print(repeat_purchases_in_categories)

# There are repeat purchases in a single category

       User_ID  Product_Category  Num_Products_in_Category
0      1000001                 1                         4
2      1000001                 3                        11
3      1000001                 4                         2
4      1000001                 5                         2
6      1000001                 8                         8
...        ...               ...                       ...
56775  1006040                 8                        62
56777  1006040                10                         2
56778  1006040                11                         7
56780  1006040                15                         4
56781  1006040                16                         4

[37900 rows x 3 columns]


## Aggregating the data to bring it into a usable format

In [ ]:
# Checking if all the demographic columns have one and only one value per User_ID
demographic_columns = ['Gender', 'Age', 'Occupation', 'City_Category', 'Marital_Status', 'Stay_In_Current_City_Years']
for col in demographic_columns:
    unique_counts = df.groupby('User_ID')[col].nunique()
    if (unique_counts > 1).any():
        print(f"{col} has multiple values for some User_IDs!")
    else:
        print(f"All User_IDs have only one unique value for {col}.")

All User_IDs have only one unique value for Gender.
All User_IDs have only one unique value for Age.
All User_IDs have only one unique value for Occupation.
All User_IDs have only one unique value for City_Category.
All User_IDs have only one unique value for Marital_Status.
All User_IDs have only one unique value for Stay_In_Current_City_Years.


In [ ]:
# Creating the numerical features
agg_df_numeric = df.groupby('User_ID').agg(
    CLV=('Purchase', 'sum'),
    Avg_Purchase_Amount=('Purchase', 'mean'),
    ProductCategory_Diversity=('Product_Category', 'nunique'),
    Num_Transactions=('Purchase', 'count')
).reset_index()
agg_df_numeric.head()

,User_ID,CLV,Avg_Purchase_Amount,ProductCategory_Diversity,Num_Transactions
0,1000001,334093,9545.514286,11,35
1,1000002,810472,10525.610390,6,77
2,1000003,341635,11780.517241,6,29
3,1000004,206468,14747.714286,2,14
4,1000005,821001,7745.292453,12,106


In [ ]:
# Converting demographic columns as features
agg_df_demographics = df.groupby('User_ID')[demographic_columns].first().reset_index()
agg_df_demographics.head()

,User_ID,Gender,Age,Occupation,City_Category,Marital_Status,Stay_In_Current_City_Years
0,1000001,F,0-17,10,A,0,2
1,1000002,M,55+,16,C,0,4+
2,1000003,M,26-35,15,A,0,3
3,1000004,M,46-50,7,B,1,2
4,1000005,M,26-35,20,A,1,1


In [ ]:
# Most Frequent Product Category
most_freq_category = (
    df.groupby('User_ID')['Product_Category']
    .agg(lambda i: i.value_counts().idxmax())
    .rename('Most_Frequent_CategoryID')
).reset_index()
most_freq_category.head()

,User_ID,Most_Frequent_CategoryID
0,1000001,3
1,1000002,1
2,1000003,1
3,1000004,1
4,1000005,8


In [ ]:
# Amount (proportion) spent on each category
amt_spend_each_category = pd.pivot_table(
    df,
    index='User_ID',
    columns='Product_Category',
    values='Purchase',
    aggfunc='sum',
    fill_value=0
)

amt_spend_each_category_prop = amt_spend_each_category.div(amt_spend_each_category.sum(axis=1), axis=0)
amt_spend_each_category_prop = amt_spend_each_category_prop.add_prefix('Cat_SpendProp_').reset_index()
amt_spend_each_category_prop.columns.name = None
amt_spend_each_category_prop.head()

,User_ID,Cat_SpendProp_1,Cat_SpendProp_2,Cat_SpendProp_3,Cat_SpendProp_4,Cat_SpendProp_5,Cat_SpendProp_6,Cat_SpendProp_7,Cat_SpendProp_8,Cat_SpendProp_9,...,Cat_SpendProp_11,Cat_SpendProp_12,Cat_SpendProp_13,Cat_SpendProp_14,Cat_SpendProp_15,Cat_SpendProp_16,Cat_SpendProp_17,Cat_SpendProp_18,Cat_SpendProp_19,Cat_SpendProp_20
0,1000001,0.184392,0.038438,0.358759,0.016798,0.047140,0.049753,0.000000,0.208152,0.0,...,0.000000,0.012553,0.0,0.032958,0.000000,0.049226,0.0,0.000000,0.0,0.001832
1,1000002,0.510405,0.019581,0.000000,0.000000,0.099312,0.111040,0.000000,0.259514,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000147
2,1000003,0.669071,0.085044,0.031923,0.000000,0.185654,0.000000,0.000000,0.017027,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.011281,0.0,0.000000
3,1000004,0.997670,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.002330
4,1000005,0.236785,0.031829,0.013217,0.009435,0.141807,0.080073,0.091296,0.317358,0.0,...,0.012848,0.000000,0.0,0.013502,0.021162,0.030688,0.0,0.000000,0.0,0.000000


In [ ]:
# Merging all the features created to form a single dataframe
dfs_to_merge = [
    agg_df_numeric,
    agg_df_demographics,
    most_freq_category,
    amt_spend_each_category_prop
]
agg_df = reduce(lambda left, right: pd.merge(left, right, on='User_ID', how='left'), dfs_to_merge)

agg_df = agg_df.reindex(columns=[
    'User_ID', 'Avg_Purchase_Amount',
    'ProductCategory_Diversity', 'Num_Transactions', 'Gender', 'Age',
    'Occupation', 'City_Category', 'Marital_Status',
    'Stay_In_Current_City_Years',
    'Most_Frequent_CategoryID', 'Cat_SpendProp_1', 'Cat_SpendProp_2',
    'Cat_SpendProp_3', 'Cat_SpendProp_4', 'Cat_SpendProp_5',
    'Cat_SpendProp_6', 'Cat_SpendProp_7', 'Cat_SpendProp_8',
    'Cat_SpendProp_9', 'Cat_SpendProp_10', 'Cat_SpendProp_11',
    'Cat_SpendProp_12', 'Cat_SpendProp_13', 'Cat_SpendProp_14',
    'Cat_SpendProp_15', 'Cat_SpendProp_16', 'Cat_SpendProp_17',
    'Cat_SpendProp_18', 'Cat_SpendProp_19', 'Cat_SpendProp_20', 'CLV'])

agg_df.head()

,User_ID,Avg_Purchase_Amount,ProductCategory_Diversity,Num_Transactions,Gender,Age,Occupation,City_Category,Marital_Status,Stay_In_Current_City_Years,...,Cat_SpendProp_12,Cat_SpendProp_13,Cat_SpendProp_14,Cat_SpendProp_15,Cat_SpendProp_16,Cat_SpendProp_17,Cat_SpendProp_18,Cat_SpendProp_19,Cat_SpendProp_20,CLV
0,1000001,9545.514286,11,35,F,0-17,10,A,0,2,...,0.012553,0.0,0.032958,0.000000,0.049226,0.0,0.000000,0.0,0.001832,334093
1,1000002,10525.610390,6,77,M,55+,16,C,0,4+,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000147,810472
2,1000003,11780.517241,6,29,M,26-35,15,A,0,3,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.011281,0.0,0.000000,341635
3,1000004,14747.714286,2,14,M,46-50,7,B,1,2,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.002330,206468
4,1000005,7745.292453,12,106,M,26-35,20,A,1,1,...,0.000000,0.0,0.013502,0.021162,0.030688,0.0,0.000000,0.0,0.000000,821001


In [ ]:
agg_df.to_csv("aggregated_walmart.csv", index=False)